## Práctica: similitud de documentos con bag-of-words y similitud del coseno

### Preparamos los datos

Se crea un método para preprocesar el texto convirtiendo las palabras en minúsculas, eliminando los signos de puntuación, las stopwords, etc..

In [ ]:

import os
import re
from docx import Document
import numpy as np
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

folder_path = './docs'  

documents = []

def preprocess_text(text):
    stop_words = set(stopwords.words('spanish'))
    text = text.lower()
    text = re.sub(r'[^a-záéíóúüñ\s]', '', text)
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 39.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 52.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/luisguillen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Se leen los documentos .docx y se almacena el contenido


In [130]:
for filename in os.listdir(folder_path):
    if filename.endswith('.docx'):
        file_path = os.path.join(folder_path, filename)
        try:
            doc = Document(file_path)
            content = ''
            for para in doc.paragraphs:
                content += para.text + ' '
            processed_content = preprocess_text(content)
            documents.append(processed_content)
        except Exception as e:
            print(f"Error al leer el archivo {filename}: {e}")

Se crea un vocabulario global (bag-of-words) y se crea un índice de vocabulario


In [131]:
vocab = set()
for doc in documents:
    vocab.update(doc.split())
    
vocab = {word: idx for idx, word in enumerate(vocab)}
vocab_size = len(vocab)

Se representa cada documento como un vector en el espacio del vocabulario

In [132]:
def doc_to_vector(doc):
    vector = np.zeros(vocab_size)
    for word in doc.split():
        if word in vocab:
            vector[vocab[word]] += 1  # Aquí contamos la aparición de cada palabra
    return vector

Creamos la matriz de documentos y calculamos la similitud

In [133]:
doc_vectors = np.array([doc_to_vector(doc) for doc in documents])

if doc_vectors.ndim == 1:
    doc_vectors = doc_vectors.reshape(1, -1)

doc_vectors = torch.tensor(doc_vectors, dtype=torch.float32)

similarity_matrix = torch.nn.functional.cosine_similarity(doc_vectors.unsqueeze(1), doc_vectors.unsqueeze(0), dim=2)

print("Matriz de similitud del coseno:")
print(similarity_matrix)

Matriz de similitud del coseno:
tensor([[1.0000, 0.0441, 0.0000, 0.0795, 0.0993, 0.1380, 0.0636, 0.0152, 0.0000,
         0.0497, 0.1462, 0.0882],
        [0.0441, 1.0000, 0.0953, 0.0000, 0.0000, 0.1027, 0.1243, 0.0339, 0.0000,
         0.1109, 0.1883, 0.1094],
        [0.0000, 0.0953, 1.0000, 0.0000, 0.0000, 0.0331, 0.1032, 0.0164, 0.0000,
         0.0895, 0.0000, 0.0530],
        [0.0795, 0.0000, 0.0000, 1.0000, 0.1001, 0.0741, 0.0577, 0.1286, 0.2166,
         0.0000, 0.1360, 0.2488],
        [0.0993, 0.0000, 0.0000, 0.1001, 1.0000, 0.0579, 0.0000, 0.1912, 0.2067,
         0.0417, 0.1132, 0.0617],
        [0.1380, 0.1027, 0.0331, 0.0741, 0.0579, 1.0000, 0.1297, 0.0354, 0.0174,
         0.0772, 0.1310, 0.1028],
        [0.0636, 0.1243, 0.1032, 0.0577, 0.0000, 0.1297, 1.0000, 0.0184, 0.0000,
         0.0600, 0.1904, 0.1540],
        [0.0152, 0.0339, 0.0164, 0.1286, 0.1912, 0.0354, 0.0184, 1.0000, 0.0862,
         0.0191, 0.0909, 0.0339],
        [0.0000, 0.0000, 0.0000, 0.2166, 0.2067,

Se identifican qué documentos son más similares entre sí y cuáles son menos similares.


In [134]:
max_similarity = 0
min_similarity = 1
most_similar_pair = None
least_similar_pair = None
for i in range(len(documents)):
    for j in range(i+1, len(documents)):
        similarity = similarity_matrix[i, j]
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_pair = (i, j)
        if similarity < min_similarity:
            min_similarity = similarity
            least_similar_pair = (i, j)

print(f"Los documentos más similares son los documentos {most_similar_pair} con una similitud de {max_similarity}")
print(f"Los documentos menos similares son los documentos {least_similar_pair} con una similitud de {min_similarity}")

Los documentos más similares son los documentos (8, 11) con una similitud de 0.3225609064102173
Los documentos menos similares son los documentos (0, 2) con una similitud de 0.0


Se muestra la cantidad de palabras en el vocabulario

In [135]:
print(f"El vocabulario tiene {vocab_size} palabras")

El vocabulario tiene 411 palabras


Por último se calculan los vectores TF-IDF de los documentos

Se usa TfidfVectorizer para convertir documentos a vectores TF-IDF


In [ ]:
import torch
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(documents).toarray()
doc_vectors = torch.tensor(doc_vectors, dtype=torch.float32)

Se calcula la matriz de similitud del coseno


In [137]:
similarity_matrix = torch.nn.functional.cosine_similarity(doc_vectors.unsqueeze(1), doc_vectors.unsqueeze(0), dim=2)

print("Matriz de similitud del coseno con TF-IDF:")
print(similarity_matrix)

Matriz de similitud del coseno con TF-IDF:
tensor([[1.0000, 0.0223, 0.0000, 0.0557, 0.0752, 0.0899, 0.0200, 0.0135, 0.0000,
         0.0241, 0.0736, 0.0416],
        [0.0223, 1.0000, 0.0811, 0.0000, 0.0000, 0.0527, 0.0624, 0.0234, 0.0000,
         0.0668, 0.1153, 0.0685],
        [0.0000, 0.0811, 1.0000, 0.0000, 0.0000, 0.0221, 0.0793, 0.0146, 0.0000,
         0.0618, 0.0000, 0.0376],
        [0.0557, 0.0000, 0.0000, 1.0000, 0.0554, 0.0488, 0.0456, 0.0894, 0.1320,
         0.0000, 0.0882, 0.1646],
        [0.0752, 0.0000, 0.0000, 0.0554, 1.0000, 0.0363, 0.0000, 0.1230, 0.1276,
         0.0275, 0.0771, 0.0348],
        [0.0899, 0.0527, 0.0221, 0.0488, 0.0363, 1.0000, 0.0629, 0.0183, 0.0083,
         0.0317, 0.0635, 0.0559],
        [0.0200, 0.0624, 0.0793, 0.0456, 0.0000, 0.0629, 1.0000, 0.0085, 0.0000,
         0.0212, 0.0895, 0.0811],
        [0.0135, 0.0234, 0.0146, 0.0894, 0.1230, 0.0183, 0.0085, 1.0000, 0.0507,
         0.0166, 0.0589, 0.0299],
        [0.0000, 0.0000, 0.0000, 0.13

Se buscan los pares de documentos más y menos similares


In [138]:
max_similarity = float('-inf')
min_similarity = float('inf')
most_similar_pair = None
least_similar_pair = None

for i in range(len(documents)):
    for j in range(i + 1, len(documents)):
        similarity = similarity_matrix[i, j].item()  # Convertir tensor a número
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_pair = (i, j)
        if similarity < min_similarity:
            min_similarity = similarity
            least_similar_pair = (i, j)

print(f"Los documentos más similares son los documentos {most_similar_pair} con una similitud de {max_similarity:.4f}")
print(f"Los documentos menos similares son los documentos {least_similar_pair} con una similitud de {min_similarity:.4f}")


Los documentos más similares son los documentos (8, 11) con una similitud de 0.2215
Los documentos menos similares son los documentos (0, 2) con una similitud de 0.0000
